# Superstore Sales Insights & Predictive Analysis

**Profile Fit:** Data Science Engineer  
This notebook performs EDA and a baseline forecasting model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

DATA = Path('..')/'data'/'Sample - Superstore.csv'

def load_data(path):
    try:
        return pd.read_csv(path, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding='latin-1')

raw = load_data(DATA)
raw.head()

In [ ]:
# Basic cleaning
raw.columns = [c.strip().replace(' ', '_').replace('-', '_') for c in raw.columns]
# Detect order date column
order_col = None
for c in raw.columns:
    if c.lower().replace('_','').startswith('orderdate'):
        order_col = c
        break
if order_col is None:
    cands = [c for c in raw.columns if 'date' in c.lower()]
    order_col = cands[0] if cands else None

if order_col:
    raw[order_col] = pd.to_datetime(raw[order_col], errors='coerce')
    raw = raw.dropna(subset=[order_col]).sort_values(order_col)

for name in ['Sales','Profit','Discount','Quantity']:
    for c in raw.columns:
        if c.lower()==name.lower():
            raw[c] = pd.to_numeric(raw[c], errors='coerce')

raw = raw.dropna(subset=[c for c in raw.columns if c.lower() in ['sales','profit']])
raw.info()

In [ ]:
# Monthly sales
monthly = raw.set_index(order_col).resample('MS')['Sales'].sum().reset_index()
plt.figure(figsize=(10,5))
plt.plot(monthly[order_col], monthly['Sales'])
plt.title('Monthly Sales Over Time')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.tight_layout()
plt.show()

In [ ]:
# Category/Sub-Category sales (choose first available)
cat_col = None
for c in raw.columns:
    if c.lower() in ['category','segment','sub_category','subcategory']:
        cat_col = c
        break

if cat_col:
    cs = raw.groupby(cat_col)['Sales'].sum().sort_values(ascending=False)
    cs.head(10)

In [ ]:
if cat_col:
    plt.figure(figsize=(8,5))
    cs = raw.groupby(cat_col)['Sales'].sum().sort_values(ascending=False)
    plt.bar(cs.index.astype(str), cs.values)
    plt.title(f'Sales by {cat_col.replace("_"," ").title()}')
    plt.xlabel(cat_col.replace('_',' ').title())
    plt.ylabel('Total Sales')
    plt.xticks(rotation=30, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Region-wise profit if available
region_col = None
for c in raw.columns:
    if c.lower() in ['region','state','city']:
        region_col = c
        break

if region_col:
    rp = raw.groupby(region_col)['Profit'].sum().sort_values(ascending=False).head(15)
    plt.figure(figsize=(10,6))
    plt.bar(rp.index.astype(str), rp.values)
    plt.title(f'Top {len(rp)} {region_col.replace("_"," ").title()}s by Profit')
    plt.xlabel(region_col.replace('_',' ').title())
    plt.ylabel('Total Profit')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Discount vs Profit
if 'Discount' in raw.columns and 'Profit' in raw.columns:
    samp = raw.sample(min(5000, len(raw)), random_state=42)
    plt.figure(figsize=(6,6))
    plt.scatter(samp['Discount'], samp['Profit'], s=10, alpha=0.6)
    plt.title('Discount vs Profit')
    plt.xlabel('Discount')
    plt.ylabel('Profit')
    plt.tight_layout()
    plt.show()

In [ ]:
# Correlation heatmap with matplotlib
num_cols = [c for c in raw.columns if raw[c].dtype.kind in 'if' and c != order_col]
if len(num_cols) >= 2:
    corr = raw[num_cols].corr()
    plt.figure(figsize=(6,5))
    plt.imshow(corr, interpolation='nearest')
    plt.title('Correlation Heatmap')
    plt.colorbar()
    plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha='right')
    plt.yticks(range(len(num_cols)), num_cols)
    plt.tight_layout()
    plt.show()

In [ ]:
# Forecasting: baseline linear regression on monthly sales
from numpy.polynomial.polynomial import polyfit, polyval
try:
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    lin = LinearRegression().fit(monthly[['Sales']].reset_index().index.values.reshape(-1,1), monthly['Sales'].values)
    X = np.arange(len(monthly)).reshape(-1,1)
    y_pred = lin.predict(X)
    r2 = r2_score(monthly['Sales'].values, y_pred)
    print('R2:', r2)
except Exception as e:
    t = np.arange(len(monthly))
    b1, b0 = polyfit(t, monthly['Sales'].values, 1)
    y_pred = b0 + b1*t

plt.figure(figsize=(10,5))
plt.plot(monthly[order_col], monthly['Sales'], label='Actual')
plt.plot(monthly[order_col], y_pred, label='Trend')
plt.title('Monthly Sales with Trend')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.legend()
plt.tight_layout()
plt.show()